In [4]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

cuda_malloc_async


In [5]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings("ignore")

In [6]:
from tensorflow.keras import datasets, layers, models, callbacks
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.regularizers import l2
from keras.layers import Conv2D
from keras.regularizers import l1
from keras.regularizers import l1_l2
from keras import regularizers

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
train_datagen_with_transforms = tf.keras.preprocessing.image.ImageDataGenerator(      #tranforming the data for data augmentation
    rescale=1. / 255,rotation_range=10, fill_mode='nearest',
    featurewise_center=True,
    featurewise_std_normalization=True,
    vertical_flip= True,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True )

train_datagen_with_no_transforms = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1.0/255)

train_generator = train_datagen_with_no_transforms.flow_from_directory(          
    "/mnt/c/Users/aniru/Desktop/train",
    target_size=(200, 200),
    batch_size=32,
    class_mode="categorical"
)
train_generator_with_data_augmentation = train_datagen_with_transforms.flow_from_directory(
    "/mnt/c/Users/aniru/Desktop/train",
    target_size=(200, 200),
    batch_size=32,
    class_mode="categorical"
)
validation_generator = val_datagen.flow_from_directory(
    "/mnt/c/Users/aniru/Desktop/test",
    target_size=(200,200),
    batch_size=32,
    class_mode="categorical"
    
)

In [9]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2) 

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_data = train_datagen.flow_from_directory("/content/drive/MyDrive/intern/train", 
                                               target_size=(200, 200), 
                                               color_mode='rgb', 
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training')

val_data = val_datagen.flow_from_directory("/content/drive/MyDrive/intern/train", 
                                           target_size=(200, 200), 
                                           color_mode='rgb', 
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

Found 1877 images belonging to 28 classes.
Found 455 images belonging to 28 classes.


In [10]:
sample_data = next(val_data)

In [11]:
sample_data[0].shape   #shape of our images

(32, 200, 200, 3)

In [12]:
nb_train_samples = train_data.samples
nb_val_samples = val_data.samples

In [13]:
train_data.samples

1877

# CNN

In [14]:
model = models.Sequential() 
model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(200,200,3), padding="same"))
model.add(keras.layers.MaxPool2D(2,2))
model.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2,2))
model.add(tf.keras.layers.Flatten())                                   
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(28,activation="Softmax"))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 200, 200, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 100, 100, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 50, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 80000)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [16]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [17]:
optimizer_fn=tf.keras.optimizers.Adam(learning_rate=0.001)

In [18]:
model.compile(optimizer=optimizer_fn,
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
history=model.fit_generator(train_data, epochs = 20, validation_data = val_data)

Epoch 1/20
43/59 [====================>.........] - ETA: 43s - loss: 3.3178 - accuracy: 0.0740

# with augmentation 

In [ ]:
model2 =models.Sequential()     
model2.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(200,200,3), padding="same"))
model2.add(tf.keras.layers.MaxPooling2D(2,2))
model2.add(tf.keras.layers.Conv2D(32, (3,3), activation="relu"))
model2.add(tf.keras.layers.Conv2D(64, (3,3), activation="relu"))
model2.add(keras.layers.MaxPooling2D(2,2))
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dropout(0.4))
model2.add(tf.keras.layers.Dense(64, activation="relu"))
model2.add(tf.keras.layers.Dense(28,activation="Softmax"))